In [2]:
import mir_eval
import librosa
import numpy as np
import argparse
import os
import glob
import demucs.separate
import csv

musdb_directory_path = "/Users/kaimikkelsen/Downloads"
separated_files_directory_path = "/Users/kaimikkelsen/Downloads/test"

In [5]:
# iterate through each directory, find the mixture file and create separated files

def get_audio_files(current_dir):
    current_dir = current_dir

    filenames = ['other.wav', 'vocals.wav', 'bass.wav', 'drums.wav']
    file_paths = []
    for filename in filenames:
        files_found = glob.glob(os.path.join(current_dir, filename))
        file_paths.extend(files_found)


    mixture_dir = os.path.join(current_dir, 'separated', 'mdx', 'mixture')
    mixture_file_paths = []

    for filename in filenames:
        files_found = glob.glob(os.path.join(mixture_dir, filename))
        mixture_file_paths.extend(files_found)


    return file_paths, mixture_file_paths


def find_mixture_wav_directories(directory):
    mixture_wav_directories = set()
    for root, dirs, files in os.walk(directory):
        if "separated" in dirs:
            pass
        else:
            for file in files:
                if file == "mixture.wav":
                    mixture_wav_directories.add(root)
                    break
    return mixture_wav_directories

mixture_wav_directories = find_mixture_wav_directories(musdb_directory_path)

if mixture_wav_directories:
    counter = 0
    for directory in mixture_wav_directories:
        print(directory)
        os.chdir(directory)
        print(f"{counter} of {len(mixture_wav_directories)}")
        demucs.separate.main(["-n", "mdx", directory+"/mixture.wav"]) #call demucs separation command
        counter = counter + 1
else:
    print("No mixture.wav files found in the directory or its subdirectories.")


No mixture.wav files found in the directory or its subdirectories.


In [ ]:
# Calculate the separation SDR values and write to a csv file

def getSeparationMetrics(audio_ref_list, audio_list):
    reference_sources = np.array(audio_ref_list)
    estimated_sources = np.array(audio_list)
    (sdr, sir, sar, perm) = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources, False)
    return np.mean(sdr), np.mean(sir), np.mean(sar)

def getSeparationMetricsFramewise(audio_ref_list, audio_list):
    reference_sources = np.array(audio_ref_list)
    estimated_sources = np.array(audio_list)
    (sdr, sir, sar, perm) = mir_eval.separation.bss_eval_sources_framewise(reference_sources, estimated_sources, window=22050, hop=11025)
    return np.mean(sdr), np.mean(sir), np.mean(sar)

def write_to_csv(data, file_path):
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Directory", "Other", "Vocals", "Bass", "Drums"])
        for row in data:
            writer.writerow(row)


directories = [d for d in os.listdir(separated_files_directory_path) if os.path.isdir(os.path.join(separated_files_directory_path, d))]

data_to_write = []
counter = 0
for directory in directories:
    reference_files, separated_files = get_audio_files(separated_files_directory_path+"/"+directory)
    print(directory)
    print(f"{counter} of {len(directories)}")
    counter = counter + 1
    
    if(reference_files and separated_files):

        row_data = [directory]
        for i in range(4):

            audio_ref, _ = librosa.load(reference_files[i])
            audio_sep, _ = librosa.load(separated_files[i])

            ref_list = [audio_ref]
            sep_list = [audio_sep]

            print(ref_list)

            sdr, sir, sar = getSeparationMetricsFramewise(ref_list, sep_list)
            print("SDR:", sdr)
            row_data.append(sdr)
        data_to_write.append(row_data)

csv_file_path = "/Users/kaimikkelsen/sdr_eval/demucs_output_framewise.csv"
write_to_csv(data_to_write, csv_file_path)
print("CSV file written successfully.")

